In [95]:
%cd ~/google_drive/code/projects/sparse/python
from sparse.core.sparse_dataframe import SparseDataFrame
from sparse.utilities.utils import *

%cd ~/Documents/projects/texture_classifier/python
# %cd /home/ubuntu/texture_classifier/python
import multiprocessing
import cPickle
import PIL
from sklearn.cluster import KMeans
from scipy.cluster import hierarchy
from scipy.spatial.distance import pdist, squareform
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
import cv2
from pandas.io.pytables import HDFStore
# from plotly import plotly
# from plotly.graph_objs import *

import core.utils
reload(core.utils)
from core.utils import *

import core.image_scanner
reload(core.image_scanner)
from core.image_scanner import ImageScanner

import core.pipeline
reload(core.pipeline)
from core.pipeline import *

import model
reload(model)
from model import *

/Users/alexbraun/google_drive/code/projects/sparse/python
/Users/alexbraun/Documents/projects/texture_classifier/python


In [ ]:
# def archive(info, hdf_path, features=['r', 'g', 'b', 'h', 's', 'v', 'fft_max', 'fft_std']):
#     hdf = HDFStore(hdf_path)
#     data = get_data(info, features=features)
#     x = data.drop('y', axis=1)
#     y = data.y
#     train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2)
#     hdf['train_x'] = DataFrame(train_x)
#     hdf['test_x'] = DataFrame(test_x)
#     hdf['train_y'] = Series(train_y)
#     hdf['test_y'] = Series(test_y)
#     hdf.close()
#     return train_x, test_x, train_y, test_y

# def majority_vote(y_true, y_pred):
#     d = DataFrame([y_true, y_pred]).T
#     d.columns = ['ytrue', 'yhat']
#     lut = {k:i for i, k in enumerate(wood_mask)}
#     ilut = {v:k for k, v in lut.iteritems()}
#     d.yhat = d.yhat.apply(lambda x: lut[x])
#     d = d.groupby('ytrue').agg(lambda x: x.mode()).yhat.apply(lambda x: ilut[x])
#     d = DataFrame(d)
#     print(d[d.index == d.yhat].shape[0] / float(d.shape[0]))
#     return d

# def predict(info, estimator):
#     y = []
#     for i, row in info.iterrows():
#         x = get_data(row.to_frame().T)
#         x = x.drop('y', axis=1)
#         x = Series(estimator.predict(x)).mode().values[0]
#         y.append(x)
#     return Series(y)

In [151]:
db_path = '/Users/alexbraun/Documents/projects/texture_classifier/web/static/database'
source = db_path + '/images'
spec = [
    'material',
    'image_id',
    'label',
    'origin',
    'desc',
    'extension'
]

# wood_mask = [
#     'olive',
#     'european-ash',
#     'african-mahogany',
#     'bamboo',
#     'sapele',
#     'teak',
#     'european-lime'
# ]

# wood_mask = [
#     'olive',
#     'european-ash',
#     'african-mahogany',
#     'bamboo',
#     'sapele',
#     'teak'
#     'european-lime'
#     'makore'
# ]

wood_mask = [
    'african-mahogany',
    'african-walnut',
    'afrormosia',
    'afzelia',
    'apple',
    'bamboo',
    'birdseye-maple',
    'black-cherry',
    'black-poplar',
    'bloodwood',
    'bog-oak',
    'bubinga',
    'cedar-of-lebanon',
    'ceylon-satinwood',
    'european-ash',
    'european-beech',
    'european-hornbeam',
    'european-larch',
    'european-lime',
    'goncalo-alves',
    'hemlock',
    'honey-locust',
    'iroko',
    'kingwood',
    'lati',
    'louro-preto',
    'macassar-ebony',
    'makore',
    'mansonia',
    'merbau',
    'moabi',
    'okoume',
    'olive',
    'olive-ash',
    'pear',
    'peruvian-walnut',
    'red-alder',
    'sapele',
    'sweet-chestnut',
    'teak',
    'tineo',
    'wenge',
    'western-red-cedar'
]

# variants = [
#     'african-mahogany',
#     'african-padauk',
#     'amazakou',
#     'american-black-walnut',
#     'american-mahogany',
#     'american-white-oak',
#     'aningeria',
#     'arariba',
#     'bamboo',
#     'beech',
#     'birch',
#     'black-cherry',
#     'black-poplar',
#     'blue-gum',
#     'ceylon-satinwood',
#     'coigue',
#     'courbaril',
#     'lati',
#     'mansonia',
#     'maple',
#     'north-carolina-pine',
#     'oak',
#     'santos-rosewood',
#     'sapele',
#     'scotch-pine',
#     'sweet-gum',
#     'western-red-cedar',
#     'whitewood',
#     'zebrano'
# ]

# wood_mask = variants

In [152]:
# random scan
# min_res = 10
# max_res = 100
# PARAMS = {
#             'scan_method':      'random_scan',
#             'min_resolution':   (min_res, min_res),
#             'max_resolution':   (max_res, max_res),
#             'patches':          100,
#             'patch_resolution': (max_res, max_res),
# #             'normalize':        True,
#             'bins':             256
# #             'rotation':         'random'
# }

# grid scan
min_res = 100
max_res = 100
PARAMS = {
            'scan_method':      'grid_scan',
            'min_resolution':   (min_res, min_res),
            'max_resolution':   (max_res, max_res),
            'resolutions':      1,
            'spacing':          'even',
#             'patch_resolution': (min_res, min_res),
            'normalize':        True,
            'bins':             256
}

info = get_info(source, spec, '\.')
# info = info[info.label.apply(lambda x: x in wood_mask)]
info['params'] = None
info.params = info.params.apply(lambda x: PARAMS)
info = info[info.desc.apply(lambda x: '_d_' in x)]
info.label = info.label.apply(lambda x: re.sub('--.*', '', x))

# train, test = info_split(info)

# train_info = info[info.desc.apply(lambda x: '_d_' in x)]
# test_info = info[info.origin == 'wood-database']i

# train_x, test_x, train_y, test_y = archive(info, hdf_path)

# info = info[info.origin != 'arroway-textures']
# info = info.head(20)

In [153]:
!rm -rf /var/tmp/hdf_batch/

In [154]:
version = '19'.zfill(3)
descriptor = '.aw-d_wood-mask.100x100.1-even.grid'
hdf_path = '/Users/alexbraun/Documents/projects/texture_classifier/data/hdf/'
hdf_path += 'data.' + version + descriptor + '.hdf'

data = get_data(info, hdf_path, processes=23)
data = pd.read_hdf(hdf_path)
X = data.drop('y', axis=1)
y = data.y
train_x, test_x, train_y, test_y = train_test_split(X, y)

indices     0 - 23    written to /var/tmp/hdf_batch/data.0000.hdf.batch
indices    23 - 46    written to /var/tmp/hdf_batch/data.0001.hdf.batch
indices    46 - 69    written to /var/tmp/hdf_batch/data.0002.hdf.batch
indices    69 - 92    written to /var/tmp/hdf_batch/data.0003.hdf.batch
indices    92 - 115   written to /var/tmp/hdf_batch/data.0004.hdf.batch
indices   115 - 137   written to /var/tmp/hdf_batch/data.0005.hdf.batch


In [155]:
clf = RandomForestClassifier()
params = {
    'n_estimators':        [100], #[1000],
#     'criterion':           ['gini'],
    'max_depth':           [None],
#     'min_samples_split':   [2],
#     'min_samples_leaf':    [1],
    'max_features':        [38], 
#     'max_leaf_nodes':      [None],
#     'bootstrap':           [True],
#     'oob_score':           [False],
    'n_jobs':              [-1]
#     'random_state':        [42]
#     'verbose':             [0],
#     'min_density':         [None],
#     'compute_importances': [None]
}

# clf = SVC()
# params = {
#     'C':            [5], #np.arange(0.1, 1, 0.1),
#     'kernel':       ['linear'],#, 'rbf'],
# #     'degree':       [3],
# #     'gamma':        [0.0],
# #     'coef0':        [0.0],
# #     'shrinking':    [True],
# #     'probability':  [False],
# #     'tol':          [0.001],
# #     'cache_size':   [200],
# #     'class_weight': [None],
# #     'verbose':      [False],
# #     'max_iter':     [1],
# #     'random_state': [None]
# }

train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2)

grid = GridSearchCV(clf, params, cv=5)
%time grid.fit(train_x, train_y)
print(grid.best_params_)
print('best_score:', grid.best_score_)
# pred = grid.best_estimator_.predict(valid_x)
# valid_report = get_report(valid_y, pred)
# valid_report

model_path = '/Users/alexbraun/Documents/projects/texture_classifier/web/static/database/models/'
model_path += 'random-forest.' + version + descriptor + '.pkl'
with open(model_path, 'w') as m:
    cPickle.dump(grid.best_estimator_, m)

pred = grid.best_estimator_.predict(test_x)
print(grid.best_estimator_.score(test_x, test_y))
report = get_report(test_y, pred)
report

CPU times: user 29min 13s, sys: 17.6 s, total: 29min 30s
Wall time: 1min 31s
{'max_features': 38, 'n_estimators': 100, 'n_jobs': -1, 'max_depth': None}
best_score: 0.98467896679
0.987011453536


,precision,recall,f1-score,support
0,,,,
african-mahogany,0.87,0.95,0.91,118
african-padauk,0.99,0.99,0.99,94
african-walnut,1.00,1.00,1.00,51
afrormosia,1.00,1.00,1.00,56
afzelia,1.00,0.98,0.99,64
amazakou,1.00,0.99,1.00,121
amburana,1.00,1.00,1.00,54
american-black-walnut,1.00,1.00,1.00,118
american-mahogany,1.00,0.96,0.98,112


In [156]:
report.sort('precision')

/Applications/anaconda/envs/dev/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,precision,recall,f1-score,support
0,,,,
african-mahogany,0.87,0.95,0.91,118
okoume,0.88,0.98,0.93,57
european-ash,0.91,0.96,0.93,50
wild-service,0.93,1.00,0.96,52
makore,0.93,0.97,0.95,59
apple-birch,0.93,0.92,0.92,59
black-alder,0.94,0.92,0.93,71
american-whitewood,0.94,0.97,0.96,67
honey-locust,0.95,0.85,0.89,65


In [142]:
txt = TextureClassifier(db_path, model_path)
results = txt.classification_report(info)
get_report(results.ytrue, results.ypred)

,precision,recall,f1-score,support
0,,,,
african-mahogany,1.00,1.00,1.00,2
african-padauk,1.00,1.00,1.00,2
african-walnut,1.00,1.00,1.00,1
afrormosia,1.00,1.00,1.00,1
afzelia,1.00,1.00,1.00,1
amazakou,1.00,1.00,1.00,2
amburana,1.00,1.00,1.00,1
american-black-walnut,1.00,1.00,1.00,2
american-mahogany,1.00,1.00,1.00,2


In [135]:
mask = results.ytrue != results.ypred
results[mask].groupby('ytrue').count()

,origin,confidence,ypred
ytrue,,,


In [190]:
g = txt.predict(info[info.label == 'iroko'].source.values[0])
g

[{'confidence': 0.9966666666666667,
  'description': [{u'content': None, u'heading': u'Iroko (Milicia excelsa)'},
   {u'content': u' Iroko', u'heading': u'Common Name(s)'},
   {u'content': u' Milicia excelsa, M. regia (syn. Chlorophora excelsa, C. regia)',
    u'heading': u'Scientific Name'},
   {u'content': u' Tropical Africa', u'heading': u'Distribution'},
   {u'content': u' 100-130 ft (30-40 m) tall, 3-5 ft (1-1.5 m) trunk diameter',
    u'heading': u'Tree Size'},
   {u'content': u'  41 lbs/ft3 (660 kg/m3)',
    u'heading': u'Average Dried Weight'},
   {u'content': u' .55, .66', u'heading': u'Specific Gravity (Basic, 12% MC)'},
   {u'content': u' 1,260 lbf (5,610 N)', u'heading': u'Janka Hardness'},
   {u'content': u' 12,700 lbf/in2 (87.6 MPa)',
    u'heading': u'Modulus of Rupture'},
   {u'content': u' 1,360,000 lbf/in2 (9.38 GPa)',
    u'heading': u'Elastic Modulus'},
   {u'content': u' 7,840 lbf/in2 (54.0 MPa)',
    u'heading': u'Crushing Strength'},
   {u'content': u' Radial: 2.

In [191]:
x = []
with open ('/Users/alexbraun/Documents/projects/texture_classifier/web/static/demo/demo.json', 'r') as w:
    x = json.load(w)

In [189]:
len(x)

5

In [33]:
pkl = '/Users/alexbraun/google_drive/code/projects/texture_classifier/web/database/models/random-forest.arroway-d.grid.100x100.pkl'
p = None
with open(pkl, 'r') as l:
    p = cPickle.load(l)

In [57]:
info = get_info(source, spec, '\.')

x = info[['origin', 'label']].groupby('origin').agg(lambda x: x.unique().tolist())
a, b, c = x.label.tolist()
sorted(set(a).intersection(c))

['african-mahogany',
 'african-walnut',
 'afrormosia',
 'afzelia',
 'apple',
 'bamboo',
 'birdseye-maple',
 'black-cherry',
 'black-poplar',
 'bloodwood',
 'bog-oak',
 'bubinga',
 'cedar-of-lebanon',
 'ceylon-satinwood',
 'european-ash',
 'european-beech',
 'european-hornbeam',
 'european-larch',
 'european-lime',
 'goncalo-alves',
 'hemlock',
 'honey-locust',
 'iroko',
 'kingwood',
 'lati',
 'louro-preto',
 'macassar-ebony',
 'makore',
 'mansonia',
 'merbau',
 'moabi',
 'okoume',
 'olive',
 'olive-ash',
 'pear',
 'peruvian-walnut',
 'red-alder',
 'sapele',
 'sweet-chestnut',
 'teak',
 'tineo',
 'wenge',
 'western-red-cedar']

In [180]:
.931 + .023 + .012 + .009

0.9750000000000001